In [1]:
import os
from pathlib import Path
import re
import time

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [2]:
# Konfiguration
input_dir = Path('data/raw')
text_dir = Path('data/text')
table_dir = Path('data/table')
tabletext_dir = Path('data/tabletext')
output_dir = Path('data/processed')
min_length = 100

# 1. Texte aus den Rohdaten extrahieren

In [3]:
# Alle .md-Dateien im Eingabeordner durchgehen
for input_file in input_dir.glob('*.md'):
    # Datei einlesen
    with input_file.open('r', encoding='utf-8') as f:
        lines = f.readlines()

    # Zeilen filtern
    filtered_lines = []
    for line in lines:
        stripped = line.strip()
        if len(stripped) >= min_length:
            filtered_lines.append(line)

    # Neuen Dateinamen erstellen
    output_filename = input_file.stem + '_text.md'
    output_file = text_dir / output_filename

    # Gefilterte Zeilen schreiben
    with output_file.open('w', encoding='utf-8') as f:
        f.writelines(filtered_lines)

# 2. Tabellen aus den Rohdaten extrahieren

In [4]:
# Alle .md-Dateien im Eingabeordner durchgehen
for input_file in input_dir.glob('*.md'):
    # Datei einlesen
    with input_file.open('r', encoding='utf-8') as f:
        lines = f.readlines()

    # Zeilen filtern: nur Tabellenbereiche behalten
    filtered_lines = []
    inside_table = False
    for line in lines:
        stripped = line.strip()
        if stripped == "--- Tabelle Start ---":
            inside_table = True
            filtered_lines.append(line)
        elif stripped == "--- Tabelle Ende ---":
            inside_table = False
            filtered_lines.append(line)
        elif inside_table:
            filtered_lines.append(line)
        # Alles außerhalb der Tabellenbereiche wird ignoriert

    # Zweiter Durchgang: Tabellen nummerieren und Endmarkierungen entfernen
    final_lines = []
    table_count = 0
    for line in filtered_lines:
        stripped = line.strip()
        if stripped == "--- Tabelle Start ---":
            table_count += 1
            final_lines.append(f"{table_count}. Tabelle\n")
        elif stripped == "--- Tabelle Ende ---":
            continue  # Zeile wird entfernt
        else:
            final_lines.append(line)

    # Neuen Dateinamen erstellen
    output_filename = input_file.stem + '_tabelle.md'
    output_file = table_dir / output_filename

    # Gefilterte und umgewandelte Zeilen schreiben
    with output_file.open('w', encoding='utf-8') as f:
        f.writelines(final_lines)

# 3. Datenstruktur für alle Tabellen erstellen

- table_data
  - continental
    - 2023
      - 1
      - 2
    - 2024
      - 1
  - bmw
    - 2023
      - 1


In [8]:
folder_path = table_dir  # z. B. "path/to/md/files"
table_data = {}          # Zentraler Speicher für alles

def extract_metadata(filename):
    parts = filename.split('_')
    firm = parts[0].lower()
    year = parts[1]
    return firm, year

for filename in os.listdir(folder_path):
    if filename.endswith(".md"):
        filepath = os.path.join(folder_path, filename)

        tables = {}
        current_table_number = None
        current_table_lines = []

        with open(filepath, "r", encoding="utf-8") as file:
            for line in file:
                match = re.match(r"(\d+)\. Tabelle", line.strip())
                if match:
                    if current_table_number is not None and current_table_lines:
                        tables[current_table_number] = ''.join(current_table_lines).strip()
                    current_table_number = int(match.group(1))
                    current_table_lines = []
                elif current_table_number is not None:
                    current_table_lines.append(line)

            # Letzte Tabelle hinzufügen
            if current_table_number is not None and current_table_lines:
                tables[current_table_number] = ''.join(current_table_lines).strip()

        # Metadaten extrahieren: Firma & Jahr
        firm_name, year = extract_metadata(filename)

        # Sicherstellen, dass Verschachtelung existiert
        if firm_name not in table_data:
            table_data[firm_name] = {}
        if year not in table_data[firm_name]:
            table_data[firm_name][year] = {}

        # Tabellen einfügen
        table_data[firm_name][year] = tables

----------



# Test

In [11]:
table = table_data['daimler']['2023'][50]
print(table)

| Absatz? | 447.790 Fahrzeuge | leicht unter Vorjahresniveau |
| --- | --- | --- |
| Anteil elektrifizierter Fahrzeuge (xEV)? | 5% | 6-8% |
| Umsatz | 20.288 Mio.€ | auf Vorjahresniveau |
| Bereinigte Umsatzrendite | 15,1% | 12-14% |
| Bereinigte Cash Conversion Rate | 1,0 | 0,6-0,8 |
| Sachinvestitionent | 351 Mio.€ | deutlich iiber Vorjahresniveau |
| Forschungs- und Entwicklungsleistungen! | 873 Mio.€ | deutlich iiber Vorjahresniveau |
| Mercedes-Benz Mobility | | |
| Neugeschaft! | 62.014 Mio.€ | leicht iiber Vorjahresniveau |
| Vertragsvolumen | 135.027 Mio.€ | auf Vorjahresniveau |
| Umsatz | 26.718 Mio. € | auf Vorjahresniveau |
| Bereinigte Eigenkapitalrendite! | 12,3% | 10-12% |


In [12]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=1,max_tokens=None,google_api_key=os.getenv("GOOGLE_API_KEY"))
prompt = f"""Erstelle zu der untenstehenden Tabelle einen Fließtext, der den Inhalt der Tabelle zusammenfasst.
        Verfasse zu jeder Dateneinheit einen eigenen, klar verständlichen Satz.
        Jeder Satz soll eigenständig sinnvoll sein, damit der Kontext auch dann erhalten bleibt, wenn man nur einen einzelnen Satz liest.
        Verwende kurze und präzise Formulierungen. Antworte nur mit dem Fließtext, ohne zusätzliche Erklärungen oder Einleitungen.

        Tabelle:
        {table}
        """

response = llm.invoke([HumanMessage(content=prompt)])
response.content

'Im betrachteten Absatz wurden 447.790 Fahrzeuge abgesetzt, was leicht unter dem Vorjahresniveau liegt. Der Anteil elektrifizierter Fahrzeuge (xEV) betrug 5%, angestrebt werden 6-8%. Der Umsatz belief sich auf 20.288 Mio.€ und lag damit auf Vorjahresniveau. Die bereinigte Umsatzrendite betrug 15,1%, das Ziel lag bei 12-14%. Die bereinigte Cash Conversion Rate betrug 1,0, angestrebt wurden 0,6-0,8. Die Sachinvestitionen beliefen sich auf 351 Mio.€ und lagen damit deutlich über dem Vorjahresniveau. Forschungs- und Entwicklungsleistungen erreichten 873 Mio.€ und lagen ebenfalls deutlich über dem Vorjahresniveau. Bei Mercedes-Benz Mobility betrug das Neugeschäft 62.014 Mio.€, was leicht über dem Vorjahresniveau liegt. Das Vertragsvolumen lag bei 135.027 Mio.€ und damit auf Vorjahresniveau. Der Umsatz von Mercedes-Benz Mobility betrug 26.718 Mio. € und entsprach dem Vorjahresniveau. Die bereinigte Eigenkapitalrendite erreichte 12,3%, das Ziel lag bei 10-12%.'

-------------

# 4. Durch LLM eine Datei pro Firma-Jahr Kombination erstellen

In der jede Tabelle der Firma-jahr Kombination textuell beschrieben ist

In [ ]:
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=1,
    max_tokens=None,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

output_dir = "data/tabletext"
os.makedirs(output_dir, exist_ok=True)

# Neue Struktur: firm -> year -> table_index -> table
for firm_name, yearly_tables in table_data.items():
    for year, tables in yearly_tables.items():
        filename = f"{firm_name}_{year}_tabletext.md"
        filepath = os.path.join(output_dir, filename)

        # Wenn Datei nicht existiert, erstelle sie mit Überschrift
        if not os.path.exists(filepath):
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(f"# Tabelletexte für {firm_name} ({year})\n\n")

        # Bestehenden Inhalt einlesen
        with open(filepath, "r", encoding="utf-8") as f:
            existing_content = f.read()

        for idx, table in tables.items():
            marker = f"Table_{idx}_text"

            # Wenn Marker schon existiert, überspringen
            if re.search(fr"^{re.escape(marker)}\s*$", existing_content, re.MULTILINE):
                continue

            # Prompt an LLM
            prompt = f"""Erstelle zu der untenstehenden Tabelle einen Fließtext, der den Inhalt der Tabelle zusammenfasst.
                    Verfasse zu jeder Dateneinheit einen eigenen, klar verständlichen Satz.
                    Jeder Satz soll eigenständig sinnvoll sein, damit der Kontext auch dann erhalten bleibt, wenn man nur einen einzelnen Satz liest.
                    Verwende kurze und präzise Formulierungen. Antworte nur mit dem Fließtext, ohne zusätzliche Erklärungen oder Einleitungen.

                    Tabelle:
                    {table}
                    """

            response = llm.invoke([HumanMessage(content=prompt)])
            result_text = response.content.strip()

            # An Datei anhängen
            with open(filepath, "a", encoding="utf-8") as f:
                f.write(f"\n{marker}\n{result_text}\n")

            time.sleep(4.5)  # Rate Limit

# 5. Dateien mit Tabellenbeschreibungen filtern

- table_n_text Einträge entfernen
- Leere Zeilen entfernen

In [ ]:
# Pfad zum Verzeichnis mit den .md-Dateien
directory = tabletext_dir

# Regulärer Ausdruck für Zeilen wie 'table_1_text', 'table_42_text' etc.
marker_pattern = re.compile(r"^table_\d+_text\s*$", re.IGNORECASE)

# Alle .md-Dateien im Verzeichnis durchgehen
for filename in os.listdir(directory):
    if filename.endswith(".md"):
        filepath = os.path.join(directory, filename)

        with open(filepath, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Nur Zeilen behalten, die weder leer noch ein table_*_text-Marker sind
        cleaned_lines = [
            line for line in lines
            if not marker_pattern.match(line.strip()) and line.strip() != ""
        ]

        # Datei überschreiben mit den bereinigten Zeilen
        with open(filepath, "w", encoding="utf-8") as f:
            f.writelines(cleaned_lines)

# 6. Text und Tabletext zu Processed Datei zusammenführen

In [ ]:
os.makedirs(output_dir, exist_ok=True)

# Alle Textdateien im text_dir durchgehen
for filename in os.listdir(text_dir):
    if filename.endswith("_text.md"):
        base = filename.replace("_text.md", "")  # z. B. bmw_2023

        # Erwarteter Tabellentext-Dateiname
        tabletext_filename = f"{base}_tabletext.md"
        tabletext_path = os.path.join(tabletext_dir, tabletext_filename)
        text_path = os.path.join(text_dir, filename)
        processed_filename = f"{base}_processed.md"
        processed_path = os.path.join(output_dir, processed_filename)

        # Nur kombinieren, wenn Tabellentext existiert
        if not os.path.exists(tabletext_path):
            print(f"Tabellentext fehlt für {base}")
            continue

        # Inhalte einlesen
        with open(text_path, "r", encoding="utf-8") as f:
            text_content = f.read()

        with open(tabletext_path, "r", encoding="utf-8") as f:
            tabletext_content = f.read()

        # Inhalte kombinieren: erst Text, dann Tabelle
        combined_content = text_content.strip() + "\n\n" + tabletext_content.strip()

        # Neue Datei schreiben
        with open(processed_path, "w", encoding="utf-8") as f:
            f.write(combined_content)